# Wan2GP A100 Native WebUI (Cloud Notebook, Run-All)

This notebook is for a generic rented cloud notebook (not Kaggle/Colab specific).

Native-mode behavior enforced by code guards:
- SDPA attention only
- compile shortcuts disabled
- runtime auto-quantization disabled
- step-skipping cache disabled
- profile forced to 1
- BF16 full-precision model override for Wan2.2 I2V A14B

Run all cells from top to bottom.

## 1) Verify GPU

You should see an A100 (or another GPU if your provider changed instance type).

In [ ]:
import subprocess

out = subprocess.check_output(
    ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
    text=True,
)
print(out)
if 'A100' not in out:
    print('WARNING: GPU is not A100. Native full-precision runs may require much more VRAM.')

## 2) Predictable directories (cloud default paths)

In [ ]:
import os
from pathlib import Path

WAN2GP_ROOT = Path(os.environ.get('WAN2GP_ROOT', '/workspace/Wan2GP')).resolve()
WAN_CACHE_ROOT = Path(os.environ.get('WAN2GP_CACHE_ROOT', '/workspace/wan2gp-cache')).resolve()

# Modal volume compatibility:
# - if /models is mounted, use it by default so predownloaded weights are reused instantly.
modal_mount = Path('/models')
default_models_cache = modal_mount if modal_mount.exists() else (WAN_CACHE_ROOT / 'models')
default_ckpts_cache = (modal_mount / 'ckpts') if modal_mount.exists() else (WAN_CACHE_ROOT / 'ckpts')

WAN_MODELS_CACHE = Path(os.environ.get('WAN_MODELS_CACHE', str(default_models_cache))).resolve()
WAN_CKPTS_CACHE = Path(os.environ.get('WAN_CKPTS_CACHE', str(default_ckpts_cache))).resolve()
WAN_PREFETCH_SUBDIR = os.environ.get('WAN_PREFETCH_SUBDIR', 'wan22').strip() or 'wan22'
WAN_PREFETCH_PATH = (WAN_CKPTS_CACHE / WAN_PREFETCH_SUBDIR).resolve()

WAN2GP_ROOT.parent.mkdir(parents=True, exist_ok=True)
WAN_CACHE_ROOT.mkdir(parents=True, exist_ok=True)
WAN_MODELS_CACHE.mkdir(parents=True, exist_ok=True)
WAN_CKPTS_CACHE.mkdir(parents=True, exist_ok=True)

print('WAN2GP_ROOT      =', WAN2GP_ROOT)
print('WAN_CACHE_ROOT   =', WAN_CACHE_ROOT)
print('WAN_MODELS_CACHE =', WAN_MODELS_CACHE)
print('WAN_CKPTS_CACHE  =', WAN_CKPTS_CACHE)
print('WAN_PREFETCH_PATH=', WAN_PREFETCH_PATH)


## 3) Clone or update repository

In [ ]:
import os
import subprocess
import time

# Change this if your patched branch is in another repo.
REPO_URL = 'https://github.com/kanaya23/Wan2GP.git'

git_env = os.environ.copy()
git_env.setdefault('GIT_TERMINAL_PROMPT', '0')
git_env.setdefault('GIT_LFS_SKIP_SMUDGE', '1')

start = time.time()
if WAN2GP_ROOT.exists() and (WAN2GP_ROOT / '.git').exists():
    print('Repository exists -> fast pull...')
    subprocess.run(['git', '-C', str(WAN2GP_ROOT), 'pull', '--ff-only', '--quiet'], check=True, env=git_env)
else:
    if WAN2GP_ROOT.exists() and not (WAN2GP_ROOT / '.git').exists():
        raise RuntimeError(f'{WAN2GP_ROOT} exists but is not a git repo. Remove it or set WAN2GP_ROOT.')
    subprocess.run(['git', 'clone', '--depth', '1', '--single-branch', REPO_URL, str(WAN2GP_ROOT)], check=True, env=git_env)

print(f'Repository ready in {time.time() - start:.1f}s')


## 4) Install dependencies

In [ ]:
import hashlib
import os
import shutil
import subprocess
import sys
import time
from pathlib import Path

env = os.environ.copy()
env.setdefault('DEBIAN_FRONTEND', 'noninteractive')
env.setdefault('PIP_DISABLE_PIP_VERSION_CHECK', '1')
env.setdefault('PIP_NO_INPUT', '1')
env.setdefault('PIP_PROGRESS_BAR', 'off')
env.setdefault('PIP_DEFAULT_TIMEOUT', '100')

env['UV_CONCURRENT_DOWNLOADS'] = '8'
env['PIP_CACHE_DIR'] = str((WAN_CACHE_ROOT / '.pip-cache').resolve())
Path(env['PIP_CACHE_DIR']).mkdir(parents=True, exist_ok=True)

def run_quiet(cmd, label, *, env=None, cwd=None):
    start = time.time()
    print(f'[start] {label}')
    proc = subprocess.run(
        cmd,
        cwd=cwd,
        env=env,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
    )
    elapsed = time.time() - start
    if proc.returncode != 0:
        tail = '\n'.join(proc.stdout.splitlines()[-120:])
        raise RuntimeError(f'{label} failed (exit {proc.returncode})\n---- output tail ----\n{tail}')
    print(f'[ok] {label} ({elapsed:.1f}s)')

def find_uv_binary():
    uv_bin = shutil.which('uv')
    return uv_bin

uv_bin = find_uv_binary()
if uv_bin:
    print(f'[speed] using uv installer: {uv_bin}')
else:
    try:
        run_quiet([sys.executable, '-m', 'pip', 'install', '--quiet', 'uv'], 'install uv helper', env=env)
        uv_bin = find_uv_binary()
        if uv_bin:
            print(f'[speed] using uv installer: {uv_bin}')
        else:
            print('[info] uv install attempted, but binary not found; using pip fallback')
    except Exception:
        print('[info] uv unavailable; using pip fallback')

apt_prefix = ['apt-get'] if os.geteuid() == 0 else ['sudo', 'apt-get']
apt_packages = ['ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2']
missing_apt = [
    pkg for pkg in apt_packages
    if subprocess.run(['dpkg', '-s', pkg], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL).returncode != 0
]
if missing_apt:
    run_quiet(apt_prefix + ['update', '-qq'], 'apt update', env=env)
    run_quiet(apt_prefix + ['install', '-y', '--no-install-recommends'] + missing_apt, f'apt install ({", ".join(missing_apt)})', env=env)
else:
    print('[skip] apt packages already installed')

req_file = WAN2GP_ROOT / 'requirements.txt'
req_hash = hashlib.sha256(req_file.read_bytes()).hexdigest()[:12]
setup_dir = WAN_CACHE_ROOT / '.setup-markers'
setup_dir.mkdir(parents=True, exist_ok=True)
marker = setup_dir / f'deps_{req_hash}_torch2.8.0_cu128.done'

if marker.exists():
    print(f'[skip] dependency marker found: {marker.name}')
else:
    run_quiet([sys.executable, '-m', 'pip', 'install', '--quiet', '--upgrade', 'pip', 'setuptools', 'wheel'], 'pip bootstrap', env=env)

    if uv_bin:
        installer = [uv_bin, 'pip', 'install', '--system', '--quiet']
    else:
        installer = [sys.executable, '-m', 'pip', 'install', '--quiet', '--prefer-binary']

    run_quiet(
        installer + [
            'torch==2.8.0', 'torchvision', 'torchaudio', 'xformers==0.0.32.post2',
            '--index-url', 'https://download.pytorch.org/whl/cu128',
        ],
        'install torch stack + xformers',
        env=env,
    )

    run_quiet(
        installer + ['-r', str(req_file), 'hydra-core', 'omegaconf', 'pyngrok', 'hf_transfer'],
        'install project dependencies',
        env=env,
    )

    marker.write_text('ok\n', encoding='utf-8')
    print(f'[ok] wrote marker: {marker.name}')

print('Dependency setup complete.')


## 4b) MatAnyOne headless backend patch (fix TkAgg) + import check


In [ ]:
import os
import subprocess
import sys
from pathlib import Path

preprocessing_init = WAN2GP_ROOT / 'preprocessing' / '__init__.py'
if not preprocessing_init.exists():
    preprocessing_init.write_text('', encoding='utf-8')

matanyone_init = WAN2GP_ROOT / 'preprocessing' / 'matanyone' / '__init__.py'
if not matanyone_init.exists():
    matanyone_init.write_text('', encoding='utf-8')

target = WAN2GP_ROOT / 'preprocessing' / 'matanyone' / 'tools' / 'interact_tools.py'
needle = "matplotlib.use('TkAgg')"
replacement = "matplotlib.use('Agg')"

if target.exists():
    text = target.read_text(encoding='utf-8')
    if replacement in text:
        print('MatAnyOne backend already set to Agg.')
    elif needle in text:
        target.write_text(text.replace(needle, replacement, 1), encoding='utf-8')
        print('Patched MatAnyOne backend: TkAgg -> Agg')
    else:
        print('TkAgg backend call not found; no patch needed.')
else:
    raise RuntimeError(f'Missing expected file: {target}')

check_env = os.environ.copy()
check_env['PYTHONPATH'] = f"{WAN2GP_ROOT}:{check_env.get('PYTHONPATH', '')}"
check_env['MPLBACKEND'] = 'Agg'

check = subprocess.run(
    [
        sys.executable,
        '-c',
        'import importlib; importlib.import_module("preprocessing.matanyone.app"); print("matanyone import ok")',
    ],
    cwd=str(WAN2GP_ROOT),
    env=check_env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

if check.returncode != 0:
    tail = '\n'.join(check.stdout.splitlines()[-120:])
    raise RuntimeError(f'MatAnyOne import check failed:\n{tail}')

print('MatAnyOne import check passed.')


## 5) Write full-precision override and verify native-mode guards

In [ ]:
import json

finetunes_dir = WAN2GP_ROOT / 'finetunes'
finetunes_dir.mkdir(parents=True, exist_ok=True)

override_payload = {
    'model': {
        'URLs': [
            'https://huggingface.co/DeepBeepMeep/Wan2.2/resolve/main/wan2.2_image2video_14B_high_mbf16.safetensors'
        ],
        'URLs2': [
            'https://huggingface.co/DeepBeepMeep/Wan2.2/resolve/main/wan2.2_image2video_14B_low_mbf16.safetensors'
        ],
        'text_encoder_folder': 'umt5-xxl',
        'text_encoder_URLs': [
            'https://huggingface.co/DeepBeepMeep/Wan2.1/resolve/main/umt5-xxl/models_t5_umt5-xxl-enc-bf16.safetensors'
        ],
        'auto_quantize': False
    }
}

override_file = finetunes_dir / 'i2v_2_2.json'
override_file.write_text(json.dumps(override_payload, indent=4), encoding='utf-8')
print('Wrote', override_file)

wgp_text = (WAN2GP_ROOT / 'wgp.py').read_text(encoding='utf-8')
required = [
    'attention_mode = "sdpa"',
    'compile = ""',
    'skip_steps_cache_type = ""',
    'quantizeTransformer = False',
    'def compute_profile(override_profile, output_type="video"):\n    return 1',
]
missing = [s for s in required if s not in wgp_text]
if missing:
    raise RuntimeError('Native-mode code guards missing in this checkout:\n' + '\n'.join(missing))

print('Native-mode guard check passed.')


## 6) Set ngrok token (required)

In [ ]:
import os

# Option A: set environment variable before notebook start.
# Option B: uncomment and paste token here once.
# os.environ['NGROK_AUTHTOKEN'] = 'YOUR_TOKEN_HERE'

token = os.environ.get('NGROK_AUTHTOKEN', '').strip()
if not token:
    raise RuntimeError('Missing NGROK_AUTHTOKEN. Set it before running the launch cell.')
print('NGROK_AUTHTOKEN detected.')

## 7) Launch Wan2GP WebUI on ngrok (keep this cell running)

In [ ]:
import json
import os
import shutil
import subprocess
import sys
import threading
import time
from pathlib import Path
from pyngrok import ngrok

try:
    from IPython.display import FileLink, display
except Exception:
    FileLink = None
    display = None

env = os.environ.copy()

def link_cache(src_dir, cache_dir):
    if src_dir.is_symlink():
        if src_dir.resolve() != cache_dir:
            src_dir.unlink()
            src_dir.symlink_to(cache_dir, target_is_directory=True)
        return
    if src_dir.exists():
        if src_dir.is_dir():
            for child in src_dir.iterdir():
                target_child = cache_dir / child.name
                if target_child.exists():
                    if child.is_dir():
                        shutil.rmtree(child)
                    else:
                        child.unlink()
                else:
                    shutil.move(str(child), str(target_child))
            src_dir.rmdir()
        else:
            src_dir.unlink()
    src_dir.symlink_to(cache_dir, target_is_directory=True)

link_cache(WAN2GP_ROOT / 'models', WAN_MODELS_CACHE)
link_cache(WAN2GP_ROOT / 'ckpts', WAN_CKPTS_CACHE)

# Ensure Wan2GP file locator checks mounted volume paths first (supports /models and /models/wan22).
config_file = WAN2GP_ROOT / 'wgp_config.json'
if config_file.exists():
    try:
        cfg = json.loads(config_file.read_text(encoding='utf-8'))
    except Exception:
        cfg = {}
else:
    cfg = {}

existing = cfg.get('checkpoints_paths', [])
new_paths = [str(WAN_CKPTS_CACHE), str(WAN_PREFETCH_PATH), 'ckpts', '.'] + [str(p) for p in existing]
seen = set()
merged_paths = []
for pth in new_paths:
    pth = str(pth).strip()
    if not pth or pth in seen:
        continue
    seen.add(pth)
    merged_paths.append(pth)
cfg['checkpoints_paths'] = merged_paths
config_file.write_text(json.dumps(cfg, indent=4), encoding='utf-8')
print('checkpoints_paths =', cfg['checkpoints_paths'])

required_prefetch_files = [
    'wan2.2_image2video_14B_high_mbf16.safetensors',
    'wan2.2_image2video_14B_low_mbf16.safetensors',
    'umt5-xxl/models_t5_umt5-xxl-enc-bf16.safetensors',
]
search_roots = [WAN_CKPTS_CACHE, WAN_PREFETCH_PATH]
resolved_prefetch = {}
missing_prefetch = []
for rel_path in required_prefetch_files:
    found_path = None
    for root in search_roots:
        candidate = (root / rel_path).resolve()
        if candidate.is_file():
            found_path = candidate
            break
    if found_path is None:
        missing_prefetch.append(rel_path)
    else:
        resolved_prefetch[rel_path] = found_path
if missing_prefetch:
    raise RuntimeError(
        'Missing prefetched Wan2.2 BF16 files in mounted ckpts; refusing to continue to avoid redownloads:\n'
        + '\n'.join(f'- {p}' for p in missing_prefetch)
        + '\nRun cloud_cpu_prefetch_wan22_to_volume.ipynb first with the same mounted volume.'
    )
print('[prefetch-check] Wan2.2 BF16 files found locally:')
for rel_path, local_path in resolved_prefetch.items():
    print(f' - {rel_path} -> {local_path}')

env['WAN_CACHE_DIR'] = str(WAN_MODELS_CACHE)
env['PYTHONPATH'] = f"{WAN2GP_ROOT}:{env.get('PYTHONPATH', '')}"
env.setdefault('HF_HOME', str(WAN_MODELS_CACHE / 'hf_home'))
env.setdefault('HUGGINGFACE_HUB_CACHE', str(WAN_MODELS_CACHE / 'hf_home' / 'hub'))
env.setdefault('TRANSFORMERS_CACHE', str(WAN_MODELS_CACHE / 'hf_home' / 'transformers'))
env.setdefault('HF_HUB_ENABLE_HF_TRANSFER', '1')
env.setdefault('HF_HUB_DISABLE_TELEMETRY', '1')
env.setdefault('HF_HUB_DISABLE_PROGRESS_BARS', '1')
env.setdefault('TQDM_DISABLE', '1')
env.setdefault('MPLBACKEND', 'Agg')
strict_offline = str(env.get('WAN_STRICT_OFFLINE', '0')).strip().lower() in {'1', 'true', 'yes', 'on'}
if strict_offline:
    env['HF_HUB_OFFLINE'] = '1'
    env['TRANSFORMERS_OFFLINE'] = '1'
    print('WAN_STRICT_OFFLINE enabled: Hugging Face/Transformers offline mode active.')


token = env.get('NGROK_AUTHTOKEN', '').strip()
ngrok.set_auth_token(token)
tunnel = ngrok.connect(addr='7860', bind_tls=True)
print('Ngrok URL:', tunnel.public_url)

cmd = [
    sys.executable,
    '-u',
    'wgp.py',
    '--listen',
    '--server-port', '7860',
    '--bf16',
]

print('Launching:', ' '.join(cmd))
process = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)

stop_event = threading.Event()

def keepalive_loop():
    while not stop_event.is_set():
        time.sleep(45)
        if stop_event.is_set():
            break
        print('[keepalive] Wan2GP still running...')

keepalive_thread = threading.Thread(target=keepalive_loop, daemon=True)
keepalive_thread.start()

try:
    for line in iter(process.stdout.readline, ''):
        if not line:
            break
        print(line, end='')
except KeyboardInterrupt:
    print('Stopping Wan2GP...')
    process.terminate()
finally:
    stop_event.set()
    try:
        process.wait(timeout=15)
    except Exception:
        pass
    keepalive_thread.join(timeout=1)
    try:
        ngrok.disconnect(tunnel.public_url)
    except Exception:
        pass
    print(f'Wan2GP stopped (return code: {process.returncode}).')

    # Build a downloadable archive at WAN2GP_ROOT/output.zip
    output_dir = WAN2GP_ROOT / 'outputs'
    archive_base = WAN2GP_ROOT / 'output'
    if output_dir.exists() and any(output_dir.iterdir()):
        archive_path = shutil.make_archive(str(archive_base), 'zip', root_dir=str(output_dir))
        print(f'Created downloadable archive: {archive_path}')
        if FileLink is not None and display is not None:
            display(FileLink(archive_path))
    elif output_dir.exists():
        print(f'Outputs folder exists but is empty: {output_dir}')
    else:
        print(f'No outputs folder found at {output_dir}; skipping archive creation.')
